# YouTube trending videos

## Projekt Eksploracji Danych

### Autorzy:
- Adrian Kotarski 127346
- Patryk Szczuczko 127215

## Wstępne przetwarzanie danych i atrybuty tekstowe
Początkowo dane z obydwu plików łączymy w jeden większy zbiór danych, dodając jednocześnie atrybut rozróżniający skąd dane pochodzą (atrybut 'country').

Następnie dodajemy dwa nowe atrybuty utworzone na podstawie istniejących już atrybutów, opisujące długość tytułu i opisu (odpowiednio: 'title_length' oraz 'description_length'). Atrybuty zostaną wykorzystane, by sprawdzić jaki jest rozkład wartości na tych atrybutach oraz czy istnieje korelacja pomiędzy tymi wartościami a popularnością video.

Ta sama motywacja była podstawą do utworzenia dwóch innych atrybutów: liczby tagów (atrybut 'number_of_tags') oraz liczby linków (atrybut 'number_of_links').

Jeśli chodzi o atrybuty 'trending_date' oraz 'publish_date' to ich struktura została ujednolicona, tzn. wszystkie dane zostały sprowadzone do informacji o dacie. Dodatkowo z atrybutu 'publish_time' zostały utworzone kolejne dwa atrybuty: rok (atrybut 'publish_time_year') oraz miesiąc (atrybut 'publish_time_month') publikacji video. Atrybuty te zostaną wykorzystane do grupowania filmów na podstawie wartości tych atrybutów i sprawdzeniu czy jest jakaś tendencja odnośnie popularności video w zależności od miesiąca opublikowania.

Kolejnymi atrybutami, które są rozszerzeniami istniejących są: stosunek liczby like'ów, dislike'ów i komentarzy do liczby wyświetleń. Pozwala to określić jak duża część oglądających angażuje się w filmik, tj. czy dużo osób oglądających pokazuje jakąkolwiek reakcję, czy może jest to duża liczba wyświetleń przypadkowych osób, które "nabijają" tylko tę statystykę. Istniejące atrybuty w postaci surowej liczby wyświetleń/like'ów/dislike'ów/komentarzy też mogą być przydatne przy analizie, być może istnieją pewne progi, po których szansa na bycie filmiku w trending znacząco rośnie.

Został utworzony również atrybut 'days_from_publish_time_to trending_date', którego wartość stanowi liczbę dni, które upłynęły od czasu opublikowania filmu, do czasu w którym film trafił do zakładki Trending (czyli jest to po prostu różnica między 'trending_date' a 'publish_time' wyrażona w dniach). Utworzenie tego atrybutu jest umotywowane tym, że filmy, które nie potrzebują dużo czasu od opublikowania ich, żeby trafiły do zakładki Trending, być może jakoś bardziej się wyróżniają i należy dokładniej przeanalizować dane takich video.

Następnym etapem naszych działań była wizualizacja danych na wykresach. Za pomocą wykresów pudełkowych zobrazowaliśmy rozkład wartości dla atrybutów: 'number_of_tags', 'number_of_links', 'title_length' oraz 'description_length'. Zrobiliśmy to zarówno na całym zbiorze danych, jak i dla podziału na kraje GB i US, aby sprawdzić czy są w tej kwestii jakieś wyraźne różnice, jeśli chodzi o porównanie video brytyjskich i amerykańskich. Okazało się, że te różnice nie są znaczące.

Sprawdziliśmy również jakie kategorie dominują w zbiorze danych. Niestety, okazało się, że filmy z przypisaną kategorią stanowią około 6% wszystkich filmów. Dlatego ciężko byłoby wysnuć jakąś prawidłowość na podstawie tylko tego atrybutu. Mimo to, na wykresie przedstawiliśmy, które kategorię filmów, są najbardziej popularne (stwierdziliśmy, że film popularny powinien mieć dużo wyświetleń, like'ów oraz komentarzy).

Jeśli chodzi o popularność filmów w zależności od miesiąca w jakim zostały one opublikowane, to dane pokazują, że filmy opublikowane od lipca do października nie cieszą się dużą liczbą wyświetleń i polubień. Z drugiej strony, liczba filmów opublikowana w tym okresie również jest bardzo mała. Biorąc te dwie kwestię pod uwagę, być może opublikowanie filmu w okresie lipiec-październik, ma większe szanse na trafienie do zakładki Trending, nawet jeśli nie będzie miało zbyt dużej liczby wyświetleń, polubień czy komentarzy. Chociaż najbardziej prawdopodobny jest tutaj brak zbalansowania przykładów przypadających na dany miesiąc, tj. dataset został głównie zbudowany w oparciu o przykłady z pozostałych miesięcy.

Sprawdziliśmy także, czy fakt wyłączenia możliwości komentowania lub oceny filmu wpływa negatywnie na jego liczbę odsłon. Według danych ze zbioru taka teza jest nieprawdziwa, jednak liczba filmów z wyłączoną opcją komentowania lub oceniania jest znikoma, więc mimo tego, że takie ograniczenie nie wpływa na liczbę odsłon to mała ilość takich filmów może sugerować, że nie są one zbyt chętnie brane do zakładki Trendings.

Ostatnią obserwacją jest przedstawienie macierzy korelacji. Wynika z niej dodatnia korelacja między kilkoma grupami atrybutów. Pierwszą grupę stanowią atrybuty: 'views', 'likes', 'dislikes' oraz 'comment_count'. Jest to dosyć zrozumiałe, gdyż logicznym wydaje się, że jeśli film będzie miał więcej odsłon to więcej użytkowników go oceni oraz skomentuje. Drugą grupą skorelowaną są atrybuty: 'comments_disabled' oraz 'ratings_disabled', co oznacza że jeśli komentowanie jest wyłączone to ocenianie również. Z macierzy można też zaobserwować fakt, że filmy które mają dużo tagów, częściej mają też dużo linków w opisie. Negatywną korelacją wykazują atrybuty 'publish_time_year' oraz 'days_from_publish_time_to_trending_date', co świadczy o tym, że filmy opublikowane wcześniej czekały dłużej na to, aby zaistnieć w zakładce Trendings. Jednak może mieć to związek z tym, że dany film pojawił się kilka razy w zakładce Trendings, a 'trending_date' to data ostatniego razu kiedy ten film się tam pojawił. Jeśli chodzi o grupy skorelowanych atrybutów, to z każdej takiej grupy do dalszego przetwarzania i uzyskiwania wiedzy będzie wybrany jeden z nich, aby taka grupa nie miała sztucznie zwiększonej wagi ważności.

Zdecydowaliśmy się też nie wykorzystywać zliczania słów w tytułach/opisach, jako że są one bezpośrednią implikacją treści filmiku jak i ogólnej działalności kanału. Nie ma więc sensu analiza i doszukiwanie się wzorców w tych danych, jeśli muszą one wynikać wyłącznie z charakteru filmiku i kanału, które zależą wyłącznie od youtubera. Sztuczna próba tworzenia contentu na podstawie losowych zlepków słów niezwiązanych z działalnością twórcy jest bezcelowa.

## Wykorzystany kod


Biblioteki:


In [ ]:
import os.path
import urllib.error
import urllib.request

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

Funkcje z opisem w komentarzu

In [ ]:
# Funkcja mapująca thumbnail_link na video_id
def map_video_id_from_thumbnail_link(thumbnail_link):
    return thumbnail_link.split(sep='/')[-2]


# Funkcja parsująca atrybut trending_date do struktury Timestamp
def trending_date_to_timestamp(trending_date):
    date_parts = list(map(int, str(trending_date).split(sep='.')))
    return pd.Timestamp(2000 + date_parts[0], date_parts[2], date_parts[1])

Wczytanie i wstępne przetworzenie danych

In [ ]:
# Załadowanie danych z pliku
data_GB = pd.read_csv("GB_videos_5p.csv", delimiter=';', encoding='latin1')
data_US = pd.read_csv("US_videos_5p.csv", delimiter=';')

# Dodanie atrybutu country, który rozróżnia dane pochodzące z US i GB
data_GB['country'] = "GB"
data_US['country'] = "US"

# Złączenie danych w jeden zbiór
data = pd.concat([data_GB, data_US])
data = data.reset_index(drop=True)

# Przypisanie nowych video_id dla wpisów z 'video_id' == #NAZWA? na podstawie thumbnail_link
data.loc[data['video_id'] == "#NAZWA?", "video_id"] = \
    data['thumbnail_link'].map(lambda thumbnail_link: map_video_id_from_thumbnail_link(thumbnail_link))

# Zmiana nazwy atrybuty na poprawną (bez spacji na końcu)
data = data.rename(columns={"description ": "description"})

# Zmiana struktury danych atrybutu trending_date
data['trending_date'] = data['trending_date'].map(lambda trending_date: trending_date_to_timestamp(trending_date))

# Utworzenie mappingów na potrzeby kolejnych atrybutów
video_id_to_trending_count_mapping = data['video_id'].value_counts().to_dict()
video_id_to_last_trending_date_mapping = data.sort_values('trending_date', ascending=False).drop_duplicates("video_id") \
    .set_index('video_id').to_dict()['trending_date']

# Usunięcie duplikatów filmów po video_id
data = data.sort_values('views').drop_duplicates("video_id").sort_index().reset_index(drop=True)

# Utworzenie atrybutów: liczba wystąpień w zakładce Trending oraz data ostatniego pojawienia się filmu w zakładce Trending
data['trending_count'] = data['video_id'].map(lambda video_id: video_id_to_trending_count_mapping[video_id])
data['last_trending_date'] = data['video_id'].map(lambda video_id: video_id_to_last_trending_date_mapping[video_id])

# Zmiana nazwy atrybutu, który tak naprawdę jest teraz pierwszą datą wystąpienia filmiku w zakładce Trending
data = data.rename(columns={"trending_date": "first_trending_date"})

Funkcja z opisem w komentarzu

In [ ]:
# Funkcja obliczająca liczbę tagów z atrybutu tags
def tags_to_number_of_tags(tags):
    return 0 if tags == "[none]" else len(str(tags).split(sep='|'))

Utworzenie nowych atrybutów

In [ ]:
# Utworzenie nowych atrybutów na podstawie już obecnych
data['likes_views_ratio'] = data['likes'] / data['views']
data['dislikes_views_ratio'] = data['dislikes'] / data['views']
data['comment_count_views_ratio'] = data['comment_count'] / data['views']

# Dodanie atrybutów: długości tytułu oraz długości opisu
data['title_length'] = data.apply(lambda row: len(row['title']), axis=1)
data['description_length'] = data.apply(lambda row: len(str(row['description'])), axis=1)

# Uproszczenie danych atrybutu publish_time - usunięcie czasu, zostawienie tylko daty
data['publish_time'] = data['publish_time'].map(lambda publish_time: pd.Timestamp(pd.Timestamp(publish_time).date()))

# Utworzenie dwóch nowych atrybutów: rok oraz miesiąc publikacji
data['publish_time_year'] = data['publish_time'].map(lambda publish_time: publish_time.year)
data['publish_time_month'] = data['publish_time'].map(lambda publish_time: publish_time.month)

# Utworzenie atrybutu, który opisuje ile czasu (w dniach) video potrzebowało aby pojawić się w proponowanych
data['days_from_publish_time_to trending_date'] = data['first_trending_date'] - data['publish_time']
data['days_from_publish_time_to trending_date'] = data['days_from_publish_time_to trending_date'].map(
    lambda timedelta: timedelta.days)

# Dodanie atrybutów: liczby tagów w atrybucie tags oraz liczba linków w opisie
data['number_of_tags'] = data['tags'].map(lambda tags: tags_to_number_of_tags(tags))
data['number_of_links'] = data['description'].map(lambda description: str(description).count("http"))

Wykresy i charakterystyka nowych atrybutów

In [ ]:
for column in ['number_of_tags', 'number_of_links', 'title_length', 'description_length']:
    # Rysowanie wykresów pudełkowych interesujących atrybutów liczbowych (pierwszy wykres - całość danych, drugi wykres -
    # dane w podziale na kraj US lub GB)
    data.boxplot(column=[column])
    data.boxplot(by='country', column=[column])
    plt.suptitle('')
    plt.show()

    # Wypisanie statystyk zwizualizowanych na wcześniejszych wykresach pudełkowych
    print(data[column].describe())
    print(data.groupby('country')[column].describe())

Funkcja przypisująca 

In [ ]:
# Funkcja wydobywająca nazwę kategorii video wraz z ID
def extract_category_with_id_from_json_items(items):
    items_str = str(items)
    category_id = items_str[items_str.index("id\': \'") + 6:items_str.index("\', \'snippet")]
    category = items_str[items_str.index("title\': \'") + 9:items_str.index("\', \'assignable")]
    return int(category_id), category

Przemapowanie kategorii

In [ ]:
# Załadowanie pliku z opisem kategorii (zawartość pliku GB zawiera się w zawartości pliku US jeśli chodzi o przypisanie nazwy kategorii do ID)
category_names = pd.read_json('US_category_id.json')

# Przemapowanie ID kategorii na jej nazwę
category_mapping = dict(
    category_names['items'].map(lambda items: extract_category_with_id_from_json_items(items)).tolist())
data['category_id'].replace(category_mapping, inplace=True)

# W związku z zastąpieniem ID kategorii przez jej nazwę to zmiana nazwy atrybutu
data = data.rename(columns={"category_id": "category"})

Funkcja z opisem w komentarzu

In [ ]:
def assign_category_from_youtube_api(video_id, category):
    if str(category) == "nan":
        values = trending_with_category.loc[trending_with_category["video_id"] == video_id]["category"].values
        if values.size != 0:
            return values[0]
    return category

Przypisanie kategorii z Youtube API

In [ ]:
# Przypisanie kategorii z Youtube API dla tych filmów, które miały pustą kategorię
trending_with_category = pd.read_csv("trending_with_category.csv", sep=";")
data["category"] = data\
    .apply(lambda row: assign_category_from_youtube_api(row["video_id"], row["category"]), axis=1)

Funkcja

In [ ]:
# Funkcja pomocnicza do wykresu liczba video z przypisaną kategorią vs liczba video bez przypisanej kategorii
def value_vs_non_value_in_category_id(category_id):
    return "No" if str(category_id) == "nan" else "Yes"

Wykresy odnośnie kategorii

In [ ]:
# Rysowanie wykresu obrazującego rozkład kategorii wśród video z przypisanymi kategoriami
category_count_plot = sns.countplot(x='category', data=data)
category_count_plot.set_xticklabels(category_count_plot.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.show()

# Rysowanie wykresu obrazującego porównanie liczby video z przypisaną kategorią do liczby video bez przypisanej kategorii
value_count = data['category'].map(lambda category_id: value_vs_non_value_in_category_id(category_id)).value_counts()
sns.barplot(value_count.index, value_count.values).set_title("Video has category")
plt.show()

# Wypisanie statystyk odnośnie liczby video z daną kategorią (lub bez)
print(data['category'].value_counts(dropna=False))

Kolejne wykresy i charakterystyki nowych atrybutów

In [ ]:
# Wykres liczby video w zależności od miesiąca w którym zostały opublikowane
sns.countplot(x='publish_time_month', data=data)
plt.show()

for group in ['publish_time_month', 'category']:
    for column in ['views', 'likes', 'comment_count']:
        # Rysowanie wykresów pudełkowych dla wybranych atrybutów
        # bez obserwacji odstających - aby wykres był czytelniejszy
        data.boxplot(by=group, column=[column], showfliers=False)
        if group == 'category':
            plt.xticks(rotation='vertical')
        plt.suptitle('')
        plt.show()

        # Wypisanie statystyk zwizualizowanych na wcześniejszych wykresach
        print(data.groupby(group)[column].describe())

# Wykres video z włączonymi komentarzami vs video z wyłączonymi komentarzami
sns.countplot(x='comments_disabled', data=data)
plt.show()

for column in ['views', 'likes']:
    # Rysowanie wykresów pudełkowych dla wybranych atrybutów
    # bez obserwacji odstających - aby wykres był czytelniejszy
    data.boxplot(by='comments_disabled', column=[column], showfliers=False)
    plt.suptitle('')
    plt.show()
    # Wypisanie statystyk zwizualizowanych na wcześniejszych wykresach
    print(data.groupby('comments_disabled')[column].describe())

# Wykres video z włączonymi ocenami vs video z wyłączonymi ocenami
sns.countplot(x='ratings_disabled', data=data)
plt.show()

for column in ['views', 'comment_count']:
    # Rysowanie wykresów pudełkowych dla wybranych atrybutów
    # bez obserwacji odstających - aby wykres był czytelniejszy
    data.boxplot(by='ratings_disabled', column=[column], showfliers=False)
    plt.suptitle('')
    plt.show()
    # Wypisanie statystyk zwizualizowanych na wcześniejszych wykresach
    print(data.groupby('ratings_disabled')[column].describe())

### Wnioski
- Liczba tagów jest podobna dla regionów US i GB i oscyluje mniej więcej w granicach 10-30.
- Liczba linków w opisie analogicznie, wynosi mniej więcej 5.
- Długość tytułu: średnio około 50 znaków
- Długość opisu: średnio nieco ponad 600.
- Dominują kategorie "rozrywka" i "muzyka", aczkolwiek jak wcześniej wspomniane filmiki z przypisaną kategorią stanowią niewielki odsetek całego zbioru danych
- W miesiącach czerwiec - październik jest bardzo mało filmów, prawdopodobnie niezbalansowany zbiór danych.
- Wyłączona możliwość oceniania nie ma większego wpływu na wyświetlenia.

Atrybuty, których nie da się wykorzystać lub są nieprzydatne:
- 'video_id' - służy tylko jako ID video i nie da się nic z tego wywnioskować
- 'channel_title' - są to nazwy własne, często nazwiska, i nie ma sensu tego analizować
- liczba wystąpień danych słów w jakichkolwiek łańcuchach znaków - youtube zawiera różnorodne filmiki, a słowa związane z nimi są ściśle powiązane z tematyką, którą obejmują, więc jeśli celem jest dostanie się do zakładki trendings, to zakładamy, że słowa związane z filmikiem będą naturalną implikacją jego treści, a nie losowym zlepkiem najczęściej występujących słów, z zupełnie innej kategorii.

### Aktualizacja do Etapu 1
- Po bardziej wnikliwym przejrzeniu zbioru danych okazało się że atrybut 'video_id' nie jest unikalny.
W związku z tym zliczyliśmy liczbę wystąpień dla każdej wartości 'video_id' i utworzyliśmy na jej podstawie
nowy atrybut 'trending_count'. Zdecydowaliśmy się usunąć wpisy z tymi samymi 'video_id' w taki sposób,
że zostawiliśmy wpis z najpóźniejszą datą w atrybucie 'trending_date' jednocześnie zmieniając jej nazwę na
'last_trending_date'. Dzięki tej operacji atrybut 'video_id' stał się unikalny.
Jednocześnie utworzyliśmy atrybut 'first_trending_date', który zawiera datę pierwszego
ukazania się filmu o danym 'video_id' w zakładce Trending.
- W ramach realizacji operacji z poprzedniego punktu został wykryty również fakt, że atrybut 'video_id'
posiadał również błedną wartość '#NAZWA?'. Jako, że w Etapie 2 zajmujemy się tylko atrybutami wizualnymi,
postanowiliśmy póki co usunąć te wpisy ze zbioru danych. Stanowią one około 1% wszystkich wpisów w zbiorze,
więc strata danych nie jest duża. Natomiast po analizie wpisów z tym błędnym 'video_id', wydaje się, że lepszym
pomysłem jest zbudowanie unikalnych 'video_id' z części atrybutu 'thumbnail_link', z których (jak wynika z naszych
obserwacji) są zbudowane również inne, prawidłowe wartości 'video_id'.

## Atrybuty wizualne

- Na początku napisaliśmy funkcję odpowiadającą za pobranie wszystkich obrazków i zapisanie ich do katalogu 'thumbnails'.
Okazało się, że nie wszystkie filmiki posiadają swoją miniaturkę.
- Przeglądając pobrane miniaturki zaobserwowaliśmy pewną prawidłowość, że przy górnych i dolnych krawędziach
występują czarne paski. Postanowiliśmy więc uciąć minaturki o 10 pikseli z góry i 10 pikseli z dołu, aby
późniejsze tworzenie atrybutów na podstawie charakterystyki pikseli nie było przekłamane.
- Następnie zostały utworzone atrybuty oparte na charakterystyce pikseli:
    - w przestrzeni RGB - średni kolor piksela (atrybuty: 'average_red', 'average_green', 'average_blue')
    oraz wartość dominująca ('mode_red', 'mode_green', 'mode_blue'),
    - w przestrzeni HSV - średnia wartość piksela (atrybuty: 'average_hue', 'average_saturation', 'average_value')
    oraz wartość dominująca ('mode_hue', 'mode_saturation', 'mode_value'), a także liczba pikseli mieszcząca się
    w danym przedziale wartości hue ('hue_red', 'hue_yellow', 'hue_green', 'hue_cyan', 'hue_blue', 'hue_magenta')
    - w skali szarości - kontrast RMS (root mean square)
- W przypadku wykrywania elementów na miniaturkach zdecydowaliśmy się na użycie gotowych pretrenowanych modeli z biblioteki Keras, tj. VGG16, ResNet, InceptionV3 i MobileNet. W zamyśle miały one posłużyć do wykrycia różnego rodzaju obiektów na miniaturce, tak aby umożliwić przeprowadzenie analizy z nimi związanej. Być może udałoby się w ten sposób wyznaczyć jakieś trendy. Niestety próby pracy z powyższymi modelami nie przyniosły żadnych rezultatów - testy pokazały, że te modele zupełnie nie radzą sobie z miniaturkami, które posiadamy. Wyniki, które zwracają są nietrafione i podawane z bardzo małą pewnością w większości przypadków, prawdopodobnie wynika to z tego, iż na miniaturkach najczęściej pojawiają się ludzie, a modele skupiają się na wykrywaniu zwierząt, elementów otoczenia czy garderoby. Wobec tego zdecydowaliśmy się pominąć w analizie wykorzystanie powyższych rozwiązań.
Prawdopodobna dodatkowa przyczyna słabych wyników to również fakt, że miniaturki są małego rozmiaru, w dodatku część obrazu stanowią czarne poziome paski, co przekłada się na efektywny obraz rozmiaru 120x68. Większość modeli obsługuje rozmiar 224x224 i rozciągnięcie obrazu do tych wymiarów wiąże się z dużymi zniekształceniami obrazu, co daje w efekcie obraz słabej jakości.
- Wykorzystaliśmy za to inny [model](https://github.com/jalajthanaki/Facial_emotion_recognition_using_Keras). Podane rozwiązanie wymagało odpowiedniej modyfikacji, gdyż oryginalnie program analizował pojedynczy plik i wyświetał obraz z naniesionymi zaznaczeniami twarzy wraz z rodzajem emocji. Kod został zmieniony tak, aby dla każdej z naszych miniaturek (już bez duplikatów) zwrócił odpowiadający wektor z liczbami wystąpień poszczególnych rodzajów emocji (jedna miniaturka może pokazywać np. dwie smutne osoby i jedną wesołą). Wyniki zostały zapisane do oddzielnego pliku .csv, aby uniknąć częstego wykonywania długotrwałych obliczeń. Ostatecznie w głównym skrypcie tworzony jest nowy atrybut z wektorem liczby wystąpień emocji na podstawie tego pliku.
### Wnioski
- Przeważający przedział kolorów, który występował na miniaturkach to czerwony. Było go około 2-krotnie więcej od przedziałów żółtego i niebieskiego, zajmujących w przybliżeniu na równi drugie miejsce co do częstości występowania. Oznacza to, że być może warto przemyśleć wykorzystanie wymienionego przedziału koloru, być może przeciętny widz reaguje na czerwone odcienie nieco pozytywniej od innych kolorów. Nie można jednak wykluczyć, że dominacja czerwonego jest po prostu spowodowana występowaniem twarzy na dużej części miniaturki. Kolory skóry prawdopodobnie najczęściej posiadają taką właśnie barwę, o ile nie są oświetlone specyficznym światłem. Należałoby tutaj przeprowadzić głębszą analizę.
- Najczęściej występującymi emocjami na miniaturkach były smutek, radość oraz neutralność. Dwukrotnie rzadziej występowały gniew i strach, zaskoczenie około 5-krotnie rzadziej. Wynikałoby z tego, że widzowie najsilniej reagują na widok 3 wcześniej wymienionych emocji i miniaturki powinny je zawierać. Należy mieć jednak na względzie, że zastosowano heurystyczny model, który wiąże się z nieuniknionymi pomyłkami. Należy traktować te wyniki z pewną dozą ostrożności.

Funkcje pobierające miniaturki

In [ ]:
# Funkcja pobierająca miniaturke w rozdzielczości 120x90 z podanego linku
def download_thumbnail(thumbnail_link, video_id):
    if not os.path.exists("thumbnails\\" + video_id + ".png"):
        try:
            resp = urllib.request.urlopen(thumbnail_link)
            image = np.asarray(bytearray(resp.read()), dtype="uint8")
            image = cv2.imdecode(image, cv2.IMREAD_COLOR)
            cv2.imwrite("thumbnails\\" + video_id + ".png", image)
        except urllib.error.HTTPError:
            pass


# Funkcja pobierająca miniaturke w rozdzielczości 480x360 z podanego linku
def download_hq_thumbnail(thumbnail_link, video_id):
    if not os.path.exists("thumbnails_hq\\" + video_id + ".png"):
        try:
            index = thumbnail_link.find("default.jpg")
            resp = urllib.request.urlopen(thumbnail_link[:index] + "hq" + thumbnail_link[index:])
            image = np.asarray(bytearray(resp.read()), dtype="uint8")
            image = cv2.imdecode(image, cv2.IMREAD_COLOR)
            cv2.imwrite("thumbnails_hq\\" + video_id + ".png", image)
        except urllib.error.HTTPError:
            pass

Pobranie miniaturek

In [ ]:
# Część kodu odpowiedzialna za pobranie miniaturek
for _, row in data.iterrows():
    download_thumbnail(row['thumbnail_link'], row['video_id'])
    download_hq_thumbnail(row['thumbnail_link'], row['video_id'])

Funkcje z komentarzem

In [ ]:
# Funkcja wczytująca i przetwarzająca miniaturkę (obicięcie 10 górnych i dolnych pikseli w celi wyeliminowania czarnych pasków)
def load_and_process_rgb_thumbnail(video_id):
    try:
        image = cv2.imread("thumbnails\\" + video_id + ".png")[10:-10]
        image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        return np.concatenate((extract_average(image), extract_mode(image),
                               extract_average(image_hsv), extract_mode(image_hsv),
                               extract_count_pixels_by_hue(image_hsv), extract_rms_contrast(image_gray)), axis=None)
    except:
        return 19 * ([float("NaN")])


# Funkcja wyliczająca średnie dla miniaturki filmu
def extract_average(image):
    return image.mean(axis=0).mean(axis=0)


# Funkcja wyliczająca dominantę dla miniaturki filmu
def extract_mode(image):
    colors, count = np.unique(image.reshape(-1, image.shape[-1]), axis=0, return_counts=True)
    return colors[count.argmax()]


# Funkcja zliczająca pixele w ramach poszczególnych przedziałów wartości hue
def extract_count_pixels_by_hue(image_hsv):
    hue = image_hsv[:, :, 0]
    red = np.size(hue[(hue < 15) | (hue >= 165)])
    yellow = np.size(hue[(hue >= 15) & (hue < 45)])
    green = np.size(hue[(hue >= 45) & (hue < 75)])
    cyan = np.size(hue[(hue >= 75) & (hue < 105)])
    blue = np.size(hue[(hue >= 105) & (hue < 135)])
    magenta = np.size(hue[(hue >= 135) & (hue < 165)])
    return red, yellow, green, cyan, blue, magenta


# Funckja wyliczająca kontrast RMS (root mean square)
def extract_rms_contrast(image_gray):
    return image_gray.std()

Utworzenie atrubytów wizualnych

In [ ]:
# Utworzenie atrvbutów wizualnych
data['average_red'], data['average_green'], data['average_blue'], \
data['mode_red]'], data['mode_green'], data['mode_blue'], \
data['average_hue'], data['average_saturation'], data['average_value'], \
data['mode_hue'], data['mode_saturation'], data['mode_value'], \
data['hue_red'], data['hue_yellow'], data['hue_green'], data['hue_cyan'], data['hue_blue'], data['hue_magenta'], \
data['rms_contrast'] = zip(*data['video_id'].map(lambda video_id: load_and_process_rgb_thumbnail(video_id)))

Wykres oraz charakterystyki tych atrybutów

In [ ]:
# Wykres pudełkowy liczby pikseli obrazka dla każdego z odcieni hue
hue_colors = ['hue_red', 'hue_yellow', 'hue_green', 'hue_cyan', 'hue_blue', 'hue_magenta']
data.boxplot(column=hue_colors)
plt.title("Liczba pikseli w danym odcieniu Hue")
plt.show()

# Wypisanie statystyk
for column in hue_colors:
    print(data[column].describe())

Utworzenie atrybutów wizualnych wyrażających emocje

In [ ]:
# Dodanie atrybutów wyrażających emocję na miniaturce ('angry','disgust','fear','happy','sad','surprise','neutral')
emotion_vectors = pd.read_csv("emotions_hq.csv", delimiter=',')
data = pd.merge(data, emotion_vectors, how='left', on='video_id')

Wykres oraz statystyki tych atrybutów

In [ ]:
# Wypisanie statystyk liczby emocji dla wszystkich obrazków w podziale na typ
emotion_count = {}
for emotion in ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']:
    count = np.count_nonzero(data[emotion] > 0)
    emotion_count[emotion] = count
    print(emotion + ": " + str(count))

# Rysowanie wykresu liczby emocji dla wszystkich obrazków w podziale na typ
plt.bar(emotion_count.keys(), emotion_count.values())
plt.title("Emotion counts")
plt.show()

Korelacja

In [ ]:
# Rysowanie wykresu korelacji atrybutów
f = plt.figure(figsize=(30, 30))
plt.matshow(data.corr(), fignum=f.number, cmap=plt.cm.get_cmap("coolwarm"))
plt.xticks(range(data.corr().shape[1]), data.corr().columns, fontsize=20, rotation=90)
plt.yticks(range(data.corr().shape[1]), data.corr().columns, fontsize=20)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=15)
plt.suptitle("Correlation", fontsize=64)
plt.show()

# Wypisanie macierzy korelacji
print(data.corr())

## Ocena atrybutów


Oprócz wstępnej analizy w etapie 1, dodajemy decyzję wraz z krótkim uzasadnieniem o wyborze atrybutów przydatnych w kolejnych etapach projektu. Niektóre atrybuty będą przydatne zarówno w zadaniu etykietowania, klasyfikacji filmów (trending/non-trending), jak i w pozyskaniu wiedzy dla klienta, większość jednakże jest istotna tylko w podzbiorze tych problemów (takie atrybuty też oczywiście zachowujemy).

✔ video_id/trending_date - metadane,

❌ title/tags/description - nie wykorzystujemy bezpośrednio tej informacji, jedynie już przetworzone dane w innych atrybutach,

❌ country_code - niezbyt istotny atrybut, analiza pokazuje, że dane dla regionów GB i US nie różnią się znacząco i nie ma większej potrzeby by wprowadzać takie rozróżnienie,

✔ views/likes/dislikes/comment_count - atrybuty, które będą przydatne przy uczeniu klasyfikatora i być może przy etykietowaniu, dla klienta większej informacji nie niosą, gdyż nie ma on na nie wpływu w momencie tworzenia materiału. Wysoka korelacja sugeruje redukcję atrybutów do jednego, ale być może po uwzględnieniu zbioru filmów non-trending zauważymy inne relacje tych atrybutów względem siebie, dlatego na ten moment zachowujemy je wszystkie.

❌ comments/ratings_disabled - nieprzydatne, jako że filmy blokujące komentarze/oceny stanowią drobny odsetek wszystkich filmów, a rozkłady wartości pozostałych atrybutów nie są od tego zależne,

✔ trending_count - atrybut potrzebny do odpowiedniego ważenia niektórych przypadków w przypadku uczenia modeli,

✔ likes/dislikes/comment_count ratio - na tym etapie nie jesteśmy w stanie powiedzieć czy atrybut na pewno się nie przyda przy klasyfikacji trending/non-trending, dlatego zachowamy atrybut do momentu, gdy będziemy znali rozkład wartości dla filmów non-trending.

✔ title_length - atrybut prawdopodobnie nieprzydatny w etykietowaniu, ale może się przydać przy klasyfikatorze, niesie również pewną wiedzę dla klienta,

✔ description_length - j/w,

❌ publish_time(year/month) - miesiąc i rok publikacji z pewnością nie mają większego znaczenia, dla klienta tym bardziej jest to bezużyteczna informacja,

❌ days_from_publish_time_to_trending_date - atrybut raczej nieprzydatny, nie można go użyć do nauki klasyfikatora, a dla klienta może stanowić jedynie ciekawostkę (gdy np. chce oszacować szanse na dostanie się na kartę trending, już PO wrzuceniu materiału),

✔ number_of_tags/links - może się przydać na późniejszym etapie projektu, w przypadku porównania z filmami non-trending, niesie również pewną wiedzą dla klienta.


✔ category_id - filmy zawierające ten atrybut stanowią mniejszość, ale atrybut będzie niezbędny przy zadaniu etykietowania, a także być może ujawni dodatkowe informacje dla klienta, gdy poszerzymy zbiór danych o filmy non-trending,

❌ video_has_category - jedyna informacja, którą ten atrybut niesie, to stosunek liczby filmów z- i bez kategorii. 

❌ publish_time_month - zakładamy że miesiąc, w którym opublikowano film nie ma znaczenia na żadnym kolejnym etapie projektu, dlatego całkowicie go odrzucamy,

❌ average/mode red/green/blue, rms contrast - atrybuty być może stanowiące jakąś wartość przy zadaniu etykietowania i klasyfikacji, ale nie dające żadnej wiedzy klientowi (uśrednione wartości w większości przypadków stanowią zbyt zdegenerowaną informację),

✔ average/mode hue/saturation/value - te atrybuty, ze względu na przestrzeń HSV, dają już pewną informację klientowi, taką jak preferowany dobór barwy i jasności, dodatkowo tak jak powyższa grupa atrybutów mogą się również przydać przy etykietowaniu i klasyfikacji,

✔ hue_red/yellow/green/cyan/blue/magenta - jak wyżej, przydatny atrybut, który pozwala ocenić, które kolory występują częściej w filmach trendujących, można to wykorzystać zarówno do zadania etykietowania, jak i w pozyskaniu wiedzy dla klienta,

✔ angry/disgust/fear/happy/sad/surprise/neutral - pokazują występujące emocje na miniaturkach i można wyznaczyć ogólny rozkład wartości dla całego zbioru. Na tym etapie projektu pozostawiamy ten atrybut, jako że może się okazać w przyszłości przydatny, gdy skonfrontujemy rozkład wartości występujący w filmach nietrendujących, co może się przełożyć na dodatkową wiedzę dla klienta. Może się też nawet przydać przy klasyfikacji lub etykietowaniu.

### Analiza korelacji

Zauważyliśmy oczywiste korelacje, występujące między atrybutami takimi jak liczba wyświetleń i liczba lajków, czy średnia i dominująca wartość piksela. Niestety nie udało się stwierdzić ciekawych, nietrywialnych korelacji. Większość korelacji, zarówno pozytywnych, jak i negatywnych zachodzi między atrybutami związanymi z kolorem. Minimalne korelacje widać między emocjami a kolorami, ale nie widać wyraźniejszych wzorców.
Korelacje views/likes/comment count od miesiąca publikacji również nie ujawniają ciekawych zależności, nie są też zbyt przydatne dla klienta. Wszystkie powyższe stwierdzenia dotyczą korelacji Pearsona.

## Uczenie pół-nadzorowane

- W zadaniu etykietowania zdecydowaliśmy się na użycie algorytmu Random Forest, ponieważ w porównaniu z innymi algorytmami z tej kategorii dobrze radzi sobie w przypadku niezbalansowanych danych, jak i jest odporny przeciwko zjawisku overfittingu. Korzystamy z rozwiązania biblioteki scikit-learn, dzięki czemu nie ma potrzeby pisania własnej implementacji. Dodatkowo parametryzacja jest intuicyjna i prosta w obsłudze. Po krótkich testach tuningujących zdecydowaliśmy się pozostać przy standardowych wartościach hiperparametrów. Niestety prawdopodobnie ze względu na niewielki rozmiar zbioru danych skuteczność nie jest wysoka, tj. accuracy na poziomie 23%, recall 33%.
- Druga metoda stanowi rozszerzenie pierwszej, tj. wykorzystuje Random Forest z uwzględnieniem wag dla poszczególnych klas. Wagi klas są tworzone za jako stosunek liczby przykładów z klasy najbardziej licznej do liczby przykładów z danej klasy. Dzięki takiemu zabiegowi karzemy klasyfikator bardziej za błędne sklasyfikowanie przykładu z klas mniejszościowych niż z klas dominujących. Ma to na celu zmniejszyć tendencyjność klasyfikatora w stosunku do klas większościowych. Metoda ta uzyskuje nieco lepsze wyniki, tj. accuracy i recall na poziomie 33%.
- Obie metody są do siebie zbliżone, zdecydowaliśmy się na wybór drugiej, ponieważ daje nieco lepsze wyniki i w teorii powinna działać lepiej, w szczególności na większych zbiorach danych. W następnym etapie możemy zweryfikować ponownie działanie obu metod w ramach sprawdzenia czy faktycznie tak jest.

Importy

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

Wczytanie i przetworzenie danych do uczenia

In [ ]:
# Załadowanie danych
data = pd.read_csv("trending.csv", sep=";")

# Z całego zbioru danych wybieramy te filmy, które mają opisaną kategorią i mają miniaturkę
data_with_category = data.loc[data["category"].notnull() & data["average_red"].notna()]

# Wyznaczony zbiór dzielimy na zbiór atrybutów i klas
features = data_with_category[data_with_category.columns.difference(
    ["category", "video_id", "first_trending_date", "title", "channel_title", "publish_time", "tags", "thumbnail_link",
     "description", "country", "last_trending_date"], sort=False)].copy()
target = data_with_category["category"].copy()

# Wyznaczamy wagi klas dla random forest
category_dict = data_with_category['category'].value_counts(dropna=True).to_dict()
max_category_count = max(category_dict.values())
category_weight = {k: max_category_count / v for k, v in category_dict.items()}

Klasyfikator

In [ ]:
# Tworzymy i trenujemy klasyfikator
random_forest = RandomForestClassifier(max_depth=10, min_samples_leaf=4, min_samples_split=10, n_estimators=300,
                                       max_features='auto', class_weight=category_weight, random_state=12)
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=12)
random_forest.fit(x_train, y_train)
y_pred = random_forest.predict(x_test)

Przedstawienie wyników

In [ ]:
# Statystyki klasyfikatora
print("Classification Report")
print(classification_report(y_test, y_pred))
print("Confusion Matrix")
print(confusion_matrix(y_test, y_pred))

## Ground truth

- W ramach tego etapu zostały pobrane dane z non-trending filmikami z tego samego okresu co oryginalny zbiór danych. Zostało to osiągnięte poprzez użycie Youtube API i przeglądanie "related videos" dla filmów trending i filtr dla odpowiedniego przedziału czasowego. Zgromadzony zbiór danych po odpowiednim przefiltrowaniu i usunięciu duplikatów jest podobnych rozmiarów co zbiór danych z filmikami trending. Zakładamy, że zbiór danych z filmikami trending jest kompletny, tj. przykłady w nim zawarte stanowią wszystkie filmiki, które były trendujące w danym przedziale czasowym. Dlatego znajdując related filmiki zakładamy, że jeśli nie mają one video_id występującego w zbiorze danych trending, to są one non-trending.

- Na pobranym zbiorze danych zostały wykonane analogiczne algorytmy przetwarzania obrazów i na tej samej zasadzie utworzone zostały dodatkowe atrybuty, które wykorzystywaliśmy do tej pory przy zbiorze trending, co pozwala na bezpośrednie porównanie wykresów z wartościami poszczególnych atrybutów dla obu zbiorów danych. Zdecydowaliśmy się odfiltrować na ten moment filmiki z kategoriami: travel & events oraz nonprofits & activism.

- W poprzednim etapie niefortunnie nie wykorzystaliśmy uczenia pół-nadzorowanego, ale pobraliśmy ground truth dla całego zbioru, by sprawdzić jak radzi sobie faktycznie stworzony klasyfikator, a także by móc porównać różnice w rozkładzie kategorii. Uzyskaliśmy trafność na poziomie 32%. Wartości precision i recall otrzymano na poziomie odpowiednio 0.31 i 0.32.

- Ze względu na ograniczone dzienne możliwości zapytań Youtube API musieliśmy w dość efektywny sposób wysyłać zapytania o related videos dla poszczególnych filmików ze zbioru trending o określonym video_id. Dlatego w końcowym zbiorze znajdują się filmiki będące "related" do około 1000 filmików ze zbioru trending (na jeden trending filmik przypadało zapytanie o 50 najbardziej zbliżonych filmików). Z tego powodu rozkłady wartości dla poszczególnych atrybutów być może różnią się nieznacznie od tego, co moglibyśmy uzyskać wysyłając zapytania np. o 5 najbardziej zbliżonych filmików dla wszystkich 8600 filmów. Liczymy, że pomimo to Youtube zapewni wystarczającą różnorodność danych dzięki zaawansowanym algorytmom szukania i bogatej bazie filmików.

Wczytanie danych i weryfikacja klasyfikatora

In [ ]:
# Ground truth
ground_truth = pd.read_csv("trending_with_category.csv", sep=";")
ground_truth = ground_truth.loc[ground_truth["category"].notnull() & ground_truth["average_red"].notna()]

features_ground_truth = ground_truth[ground_truth.columns.difference(
    ["category", "video_id", "first_trending_date", "title", "channel_title", "publish_time", "tags", "thumbnail_link",
     "description", "country", "last_trending_date"], sort=False)].copy()
target_ground_truth = ground_truth["category"].copy()

train_x, test_x, train_y, test_y = train_test_split(features_ground_truth, target_ground_truth, test_size=0.000000001, random_state=12)

pred_ground_truth = random_forest.predict(train_x)


Wypisanie statystyk

In [ ]:
# Statystyki klasyfikatora
print("Classification Report")
print(classification_report(train_y, pred_ground_truth))
print("Confusion Matrix")
print(confusion_matrix(train_y, pred_ground_truth))

## Przygotowanie danych non-trending

### Porównanie różnic wartości atrybutów w zbiorach trending i non-trending
Porównując liczbę tagów widzimy, że filmiki trending mają ich nieco więcej, średnio około 19, podczas gdy non-trending mają tę wartość bliżej 17, co oczywiście nie stanowi zbyt dużej różnicy i może to być kwestia wariancji z powodu niedostatecznie dużych zbiorów danych.
Rozkład liczby linków jest dość zbliżony, aczkolwiek średnia w przypadku filmików trending jest nieznacznie wyższa (około 2-3)
Jeśli chodzi o długość tytułu, to filmiki trending miały go średnio krótszy o około 5 znaków. W przypadku długości opisu różnica jest pomijalnie mała.
W przypadku miesiąca publikacji rozkład jest podobny (takie było zamierzenie), jedynie dla grudnia liczba filmów non-trending jest sporo większa od trending.
W przypadku liczby wyświetleń filmy non-trending zdają się mieć ich średnio nieco więcej. Być może wynika to z tego, w jaki sposób Youtube pokazuje related videos. Prawdopodobnie faworyzowane są filmy z większą liczbą wyświetleń.
Liczba lajków jest średnio trochę większa przy filmach trending. Liczba komentarzy ma podobny rozkład wartości dla obu wersji. Zaskakująco rozkład kolorów pikseli dla obu zbiorów jest bardzo zbliżony, liczyliśmy że w tej sferze zauważymy pewne różnice. Charaktery wykresów korelacji są bardzo podobne. W przypadku rozkładu wartości emocji występujących na miniaturkach możemy zauważyć zbliżone wykresy, aczkolwiek dla filmów trending emocja "happy" pojawia się nieco częściej, w pozostałych nie widać większych różnic. W rozkładzie kategorii widać sporo różnic, najważniejsze z nich:
- wśród filmów trending kategoria muzyka występuje znacznie częściej niż w zbiorze non-trending. 
- kategoria gaming jest znacznie częściej widoczna w zbiorze non-trending
- kategoria sport stanowi większy odsetek w filmach non-trending
- kategoria rozrywka jest reprezentowana w obu zbiorach na podobnym poziomie
- kategoria people & blogs jest mniej reprezentowana w zbiorze trending niż w zbiorze non-trending
- kategoria comedy znacznie częściej widoczna w grupie trending
- kategoria film & animation nieco rzadziej widoczna w zbiorze non-trending


Funkcje

In [ ]:
# Funkcja pobierająca miniaturke w rozdzielczości 120x90 z podanego linku
def download_non_trending_thumbnail(thumbnail_link, video_id):
    if not os.path.exists("non_trending_thumbnails\\" + video_id + ".png"):
        try:
            resp = urllib.request.urlopen(thumbnail_link)
            image = np.asarray(bytearray(resp.read()), dtype="uint8")
            image = cv2.imdecode(image, cv2.IMREAD_COLOR)
            cv2.imwrite("non_trending_thumbnails\\" + video_id + ".png", image)
        except urllib.error.HTTPError:
            pass


# Funkcja pobierająca miniaturke w rozdzielczości 480x360 z podanego linku
def download_non_trending_hq_thumbnail(thumbnail_link, video_id):
    if not os.path.exists("non_trending_thumbnails_hq\\" + video_id + ".png"):
        try:
            index = thumbnail_link.find("default.jpg")
            resp = urllib.request.urlopen(thumbnail_link[:index] + "hq" + thumbnail_link[index:])
            image = np.asarray(bytearray(resp.read()), dtype="uint8")
            image = cv2.imdecode(image, cv2.IMREAD_COLOR)
            cv2.imwrite("non_trending_thumbnails_hq\\" + video_id + ".png", image)
        except urllib.error.HTTPError:
            pass

Wczytanie danhych

In [ ]:
# Załadowanie danych z pliku
data = pd.read_csv("video_from_youtube_data_api.csv", delimiter=';')
data = data.reset_index(drop=True)

Pobranie miniaturek

In [ ]:
# Część kodu odpowiedzialna za pobranie miniaturek
for _, row in data.iterrows():
    download_non_trending_thumbnail(row['thumbnail_link'], row['video_id'])
    download_non_trending_hq_thumbnail(row['thumbnail_link'], row['video_id'])

Utworzenie nowych atrybutów

In [ ]:
# Utworzenie nowych atrybutów na podstawie już obecnych
data['likes_views_ratio'] = data['likes'] / data['views']
data['dislikes_views_ratio'] = data['dislikes'] / data['views']
data['comment_count_views_ratio'] = data['comment_count'] / data['views']

# Dodanie atrybutów: długości tytułu oraz długości opisu
data['title_length'] = data.apply(lambda row: len(row['title']), axis=1)
data['description_length'] = data.apply(lambda row: len(str(row['description'])), axis=1)

data["publish_time"] = data["publish_time"].map(
    lambda publish_time: pd.Timestamp(pd.Timestamp(publish_time).date()))

# Utworzenie dwóch nowych atrybutów: rok oraz miesiąc publikacji
data['publish_time_year'] = data['publish_time'].map(lambda publish_time: publish_time.year)
data['publish_time_month'] = data['publish_time'].map(lambda publish_time: publish_time.month)

# Dodanie atrybutów: liczby tagów w atrybucie tags oraz liczba linków w opisie
data['number_of_tags'] = data['tags'].map(lambda tags: tags_to_number_of_tags(tags))
data['number_of_links'] = data['description'].map(lambda description: str(description).count("http"))

Rysowanie wykresów i wypisanie statystyk tych powstałych atrybutów

In [ ]:
for column in ['number_of_tags', 'number_of_links', 'title_length', 'description_length']:
    # Rysowanie wykresów pudełkowych interesujących atrybutów liczbowych (pierwszy wykres - całość danych)
    data.boxplot(column=[column])
    plt.suptitle('')
    plt.show()

    # Wypisanie statystyk zwizualizowanych na wcześniejszych wykresach pudełkowych
    print(data[column].describe())

Przemapowanie ID kategorii na nazwę

In [ ]:
# Załadowanie pliku z opisem kategorii (zawartość pliku GB zawiera się w zawartości pliku US jeśli chodzi o przypisanie nazwy kategorii do ID)
category_names = pd.read_json('US_category_id.json')

# Przemapowanie ID kategorii na jej nazwę
category_mapping = dict(
    category_names['items'].map(lambda items: extract_category_with_id_from_json_items(items)).tolist())
data['category_id'].replace(category_mapping, inplace=True)

# W związku z zastąpieniem ID kategorii przez jej nazwę to zmiana nazwy atrybutu
data = data.rename(columns={"category_id": "category"})

Rysowanie wykresów i wypisanie statystyk odnośnie kategorii

In [ ]:
# Rysowanie wykresu obrazującego rozkład kategorii wśród video z przypisanymi kategoriami
category_count_plot = sns.countplot(x='category', data=data)
category_count_plot.set_xticklabels(category_count_plot.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.show()

# Rysowanie wykresu obrazującego porównanie liczby video z przypisaną kategorią do liczby video bez przypisanej kategorii
value_count = data['category'].map(lambda category_id: value_vs_non_value_in_category_id(category_id)).value_counts()
sns.barplot(value_count.index, value_count.values).set_title("Video has category")
plt.show()

# Wypisanie statystyk odnośnie liczby video z daną kategorią (lub bez)
print(data['category'].value_counts(dropna=False))

Rysowanie wykresów i wypisanie statystyk dla pozostałych atrybutów

In [ ]:
# Wykres liczby video w zależności od miesiąca w którym zostały opublikowane
sns.countplot(x='publish_time_month', data=data)
plt.show()

for group in ['publish_time_month', 'category']:
    for column in ['views', 'likes', 'comment_count']:
        # Rysowanie wykresów pudełkowych dla wybranych atrybutów
        # bez obserwacji odstających - aby wykres był czytelniejszy
        data.boxplot(by=group, column=[column], showfliers=False)
        if group == 'category':
            plt.xticks(rotation='vertical')
        plt.suptitle('')
        plt.show()

        # Wypisanie statystyk zwizualizowanych na wcześniejszych wykresach
        print(data.groupby(group)[column].describe())

# Wykres video z włączonymi komentarzami vs video z wyłączonymi komentarzami
sns.countplot(x='comments_disabled', data=data)
plt.show()

for column in ['views', 'likes']:
    # Rysowanie wykresów pudełkowych dla wybranych atrybutów
    # bez obserwacji odstających - aby wykres był czytelniejszy
    data.boxplot(by='comments_disabled', column=[column], showfliers=False)
    plt.suptitle('')
    plt.show()
    # Wypisanie statystyk zwizualizowanych na wcześniejszych wykresach
    print(data.groupby('comments_disabled')[column].describe())

# Wykres video z włączonymi ocenami vs video z wyłączonymi ocenami
sns.countplot(x='ratings_disabled', data=data)
plt.show()

for column in ['views', 'comment_count']:
    # Rysowanie wykresów pudełkowych dla wybranych atrybutów
    # bez obserwacji odstających - aby wykres był czytelniejszy
    data.boxplot(by='ratings_disabled', column=[column], showfliers=False)
    plt.suptitle('')
    plt.show()
    # Wypisanie statystyk zwizualizowanych na wcześniejszych wykresach
    print(data.groupby('ratings_disabled')[column].describe())

Utworzenie atrybutów wizualnych

In [ ]:
# Utworzenie atrvbutów wizualnych
data['average_red'], data['average_green'], data['average_blue'], \
data['mode_red]'], data['mode_green'], data['mode_blue'], \
data['average_hue'], data['average_saturation'], data['average_value'], \
data['mode_hue'], data['mode_saturation'], data['mode_value'], \
data['hue_red'], data['hue_yellow'], data['hue_green'], data['hue_cyan'], data['hue_blue'], data['hue_magenta'], \
data['rms_contrast'] = zip(*data['video_id'].map(lambda video_id: load_and_process_rgb_thumbnail(video_id)))

Rysowanie wykresów i wypisanie statystyk dla tych atrybutów

In [ ]:
# Wykres pudełkowy liczby pikseli obrazka dla każdego z odcieni hue
hue_colors = ['hue_red', 'hue_yellow', 'hue_green', 'hue_cyan', 'hue_blue', 'hue_magenta']
data.boxplot(column=hue_colors)
plt.title("Liczba pikseli w danym odcieniu Hue")
plt.show()

# Wypisanie statystyk
for column in hue_colors:
    print(data[column].describe())

Utworzenie atrybutów wyrażających emocje

In [ ]:
# Dodanie atrybutów wyrażających emocję na miniaturce ('angry','disgust','fear','happy','sad','surprise','neutral')
emotion_vectors = pd.read_csv("non_trending_emotions_hq.csv", delimiter=',')
data = pd.merge(data, emotion_vectors, how='left', on='video_id')

Rysowanie wykresów i wypisanie statystyk dla atrybutów związanych z wyrażaniem emocji

In [ ]:
# Wypisanie statystyk liczby emocji dla wszystkich obrazków w podziale na typ
emotion_count = {}
for emotion in ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']:
    count = np.count_nonzero(data[emotion] > 0)
    emotion_count[emotion] = count
    print(emotion + ": " + str(count))

# Rysowanie wykresu liczby emocji dla wszystkich obrazków w podziale na typ
plt.bar(emotion_count.keys(), emotion_count.values())
plt.title("Emotion counts")
plt.show()

Korelacja

In [ ]:
# Rysowanie wykresu korelacji atrybutów
f = plt.figure(figsize=(30, 30))
plt.matshow(data.corr(), fignum=f.number, cmap=plt.cm.get_cmap("coolwarm"))
plt.xticks(range(data.corr().shape[1]), data.corr().columns, fontsize=20, rotation=90)
plt.yticks(range(data.corr().shape[1]), data.corr().columns, fontsize=20)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=15)
plt.suptitle("Correlation", fontsize=64)
plt.show()

# Wypisanie macierzy korelacji
print(data.corr())

## Klasyfikator trending vs non-trending

Ze względu na to, że wśród atrybutów, które uwzględniamy ostatecznie dominują te o charakterze liczbowym, zdecydowaliśmy się użyć prostszych algorytmów machine learningowych, dzięki czemu nie ma również problemów z wyjaśnialnością modelów. Uznaliśmy, że porównamy ze sobą skuteczność dwóch klasyfikatorów opartych o metody odpowiednio Random Forest i MLP (Multilayer perceptron). Zbiór danych, ze względu na podobną liczbę przykładów uczących w zbiorach zarówno trending, jak i non-trending, został zwyczajnie połączony, dzięki czemu ominęliśmy problem z niezbalansowanymi danymi. Zbiór został podzielony na część treningową i testową standardową proporcją 80/20. Wyniki klasyfikatorów:

Random Forest: 
- precision: 0.76
- recall: 0.76
- f1 score: 0.76

MLP: 
- precision: 0.56
- recall: 0.53
- f1 score: 0.51

Klasyfikator w oparciu o Random Forest daje w każdym przypadku lepsze rezultaty, dlatego zdecydowaliśmy się go wybrać jako główny klasyfikator.

Załadowanie i przygotowanie danych do uczenia

In [ ]:
# Załadowanie danych
trending = pd.read_csv("trending.csv", sep=";")
non_trending = pd.read_csv("non_trending.csv", sep=";")

# Dodanie klasy do danych
trending["is_trending"] = True
non_trending["is_trending"] = False

# Połączenie zbioru danych w jeden
data = pd.concat([trending, non_trending], sort=False)
data = data.reset_index(drop=True)

# Z całego zbioru danych wybieramy te filmy, które mają miniaturkę
data = data.loc[data["average_red"].notna()]
data = data.loc[data["likes_views_ratio"].notna()]

# Wyznaczony zbiór dzielimy na zbiór atrybutów i klas
features = data[data.columns.difference(
    ["category", "video_id", "first_trending_date", "title", "channel_title", "publish_time", "tags", "thumbnail_link",
     "description", "country", "last_trending_date", "is_trending", "video_error_or_removed", "trending_count",
     "days_from_publish_time_to trending_date"], sort=False)].copy()
target = data["is_trending"].copy()

Utworzenie i trenowanie pierwszego klasyfikatora

In [ ]:
# Tworzymy i trenujemy klasyfikator
random_forest = RandomForestClassifier(max_depth=10, min_samples_leaf=4, min_samples_split=10, n_estimators=100,
                                       max_features='auto', random_state=12)
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=12)
random_forest.fit(x_train, y_train)
y_pred = random_forest.predict(x_test)

Wypisanie wyników dla pierwszego klasyfikatora

In [ ]:
# Statystyki klasyfikatora
print("Classification Report")
print(classification_report(y_test, y_pred))
print("Confusion Matrix")
print(confusion_matrix(y_test, y_pred))

Utworzenie i trenowanie drugiego klasyfikatora

In [ ]:
mlp = MLPClassifier(alpha=1, max_iter=1000, random_state=12)
mlp.fit(x_train, y_train)
y_pred = mlp.predict(x_test)

Wypisanie wyników dla drugiego klasyfikatora

In [ ]:
# Statystyki klasyfikatora
print("Classification Report")
print(classification_report(y_test, y_pred))
print("Confusion Matrix")
print(confusion_matrix(y_test, y_pred))

## Wnioski i wiedza dla klienta
W przypadku wiedzy dla klienta, należy mieć na uwadze, że wnioski są niestety dość okrojone z powodu nieumiejętnego wyboru atrybutów (w szczególności pominięcie analizy wartości tekstowych z użyciem odpowiednich modeli). Wszelkie różnice, które zaobserwowaliśmy w statystykach filmów nie są szczególnie przydatne, jako że są efektem wrzuconego do sieci filmiku, tak więc należy się skupić jedynie na atrybutach, na które ma się wpływ jeszcze przed finalną fazą wypuszczania materiału. Liczyliśmy, że zobaczymy pewne różnice w rozkładzie wartości kolorów występujących w miniaturkach, są one jednak bardzo podobne. Najwięcej informacji udało się wydobyć porównując rozkłady wartości kategorii filmików w obu zbiorach. Najważniejsze różnice jakie zaobserwowaliśmy to przewaga filmików typu gaming w zbiorze non-trending oraz fakt, że kategorie komedia i muzyka stanowiły dużo większy odsetek kategorii wśród filmów trending. Podsumowując, najważniejsze informacje, które udało nam się wydobyć sugerują, aby klient starał się wykorzystać muzyczną strefę youtube'a (nawet jeśli, jego materiały nie byłyby normalnie z tym związane, to mógłby od czasu do czasu nagrać np. jakąś muzyczną parodię - co statystycznie powinno mieć większe szanse powodzenia ze względu na większą reprezentację filmów typu comedy w zbiorze trending. Dodatkowo warto mieć na uwadze, że pozytywne emocje i radość widoczna na miniaturkach również powinna zwiększyć szanse na dostanie się do zakładki trending videos. Istnieje spora szansa, że lepiej unikać tworzenia filmików z kategorii gaming.
